<a href="https://colab.research.google.com/github/zyren123/LLM/blob/main/langchain_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import subprocess
import shlex
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pwd

/content


In [12]:
!git clone --recursive https://github.com/chatchat-space/Langchain-Chatchat.git

Cloning into 'Langchain-Chatchat'...
remote: Enumerating objects: 9016, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 9016 (delta 190), reused 225 (delta 127), pack-reused 8688
Receiving objects: 100% (9016/9016), 69.45 MiB | 31.36 MiB/s, done.
Resolving deltas: 100% (5451/5451), done.
Submodule 'knowledge_base/samples/content/wiki' (https://github.com/chatchat-space/Langchain-Chatchat.wiki.git) registered for path 'knowledge_base/samples/content/wiki'
Cloning into '/content/Langchain-Chatchat/knowledge_base/samples/content/wiki'...
remote: Enumerating objects: 565, done.        
remote: Total 565 (delta 0), reused 0 (delta 0), pack-reused 565        
Receiving objects: 100% (565/565), 117.39 KiB | 5.33 MiB/s, done.
Resolving deltas: 100% (354/354), done.
Submodule path 'knowledge_base/samples/content/wiki': checked out '2f24adb218f23eab00d7fcd7ccf5072f2f35cb3c'


In [13]:
%cd /content/Langchain-Chatchat
!pwd
! git lfs install
! git clone https://huggingface.co/zhr6212/GLM_sft_Lora



/content/Langchain-Chatchat
/content/Langchain-Chatchat
Updated git hooks.
Git LFS initialized.
Cloning into 'GLM_sft_Lora'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (20/20), 26.71 KiB | 3.82 MiB/s, done.
Filtering content: 100% (6/6), 11.63 GiB | 47.74 MiB/s, done.


In [27]:
!sed -i 's/^torch/# &/' requirements.txt
!sed -i 's/^torchvision/# &/' requirements.txt
!sed -i 's/^torchaudio/# &/' requirements.txt
!pip3 install -r requirements.txt


In [31]:
!sed -i 's#\<THUDM/chatglm3-6b\>#/content/Langchain-Chatchat/GLM_sft_Lora#' /content/Langchain-Chatchat/configs/model_config.py.example
!sed -i 's#\<chatglm3-6b\>#GLM_sft_Lora#g' /content/Langchain-Chatchat/configs/model_config.py.example

In [20]:
# %cd /content/Langchain-Chatchat/knowledge_base/samples/content/llm
!rm -rf /content/Langchain-Chatchat/knowledge_base/samples/content/test_files/*
!rm -rf /content/Langchain-Chatchat/knowledge_base/samples/content/wiki/*

In [ ]:
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

/bin/bash: line 1: conda: command not found


In [ ]:
# ! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [21]:
import torch

# 如果有GPU可用，且PyTorch与CUDA兼容，将输出CUDA设备信息
print(torch.cuda.is_available())


True


In [16]:
print(torch.version.cuda)
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)
import torchaudio
print(torchaudio.__version__)

12.1
2.1.0+cu121
0.16.0+cu121
2.1.0+cu121


In [22]:
!python -m pip install paddlepaddle-gpu==2.6.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.8/749.8 MB 941.6 kB/s eta 0:00:00


In [23]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
# !curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok config add-authtoken 2YgfRwaI3GHwnY58VdLGAYWndn5_2LfP9coSjisqWvz5AbX7h
ngrok_command='ngrok http 8501 --domain=terrier-accurate-eagerly.ngrok-free.app '

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [3,407 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,332 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,307 kB]
Hit:13 

In [24]:
! python copy_config_example.py
! conda install faiss-gpu -c conda-forge
# !cp /kaggle/input/chatglm3/model_config.py /kaggle/working/Langchain-Chatchat/configs/
# !cp /kaggle/input/chatglm3/server_config.py /kaggle/working/Langchain-Chatchat/configs/
! python copy_config_example.py
! python init_database.py --recreate-vs

/bin/bash: line 1: conda: command not found
recreating all vector stores
2024-01-05 12:10:00,626 - faiss_cache.py[line:94] - INFO: loading vector store in 'samples/vector_store/bge-large-zh' from disk.
2024-01-05 12:10:02,051 - SentenceTransformer.py[line:66] - INFO: Load pretrained SentenceTransformer: BAAI/bge-large-zh
.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 5.93MB/s]
1_Pooling/config.json: 100% 191/191 [00:00<00:00, 820kB/s]
README.md: 100% 27.9k/27.9k [00:00<00:00, 62.0MB/s]
config.json: 100% 941/941 [00:00<00:00, 3.65MB/s]
config_sentence_transformers.json: 100% 124/124 [00:00<00:00, 446kB/s]
model.safetensors: 100% 1.30G/1.30G [00:10<00:00, 120MB/s]
pytorch_model.bin: 100% 1.30G/1.30G [00:07<00:00, 170MB/s]
sentence_bert_config.json: 100% 52.0/52.0 [00:00<00:00, 278kB/s]
special_tokens_map.json: 100% 125/125 [00:00<00:00, 554kB/s]
tokenizer.json: 100% 439k/439k [00:00<00:00, 3.34MB/s]
tokenizer_config.json: 100% 366/366 [00:00<00:00, 1.84MB/s]
vocab.txt: 100% 110k/110k [00

In [25]:
ngrok_process = subprocess.Popen(shlex.split(ngrok_command), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
! python startup.py -a



==============================Langchain-Chatchat Configuration==============================
操作系统：Linux-6.1.58+-x86_64-with-glibc2.35.
python版本：3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
项目版本：v0.2.9
langchain版本：0.0.352. fastchat版本：0.2.34


当前使用的分词器：ChineseRecursiveTextSplitter
当前启动的LLM模型：['GLM_sft_Lora', 'zhipu-api', 'openai-api'] @ cuda
{'device': 'cuda',
 'host': '0.0.0.0',
 'infer_turbo': False,
 'model_path': '/content/Langchain-Chatchat/GLM_sft_Lora',
 'model_path_exists': True,
 'port': 20002}
{'api_key': '',
 'device': 'auto',
 'host': '0.0.0.0',
 'infer_turbo': False,
 'online_api': True,
 'port': 21001,
 'provider': 'ChatGLMWorker',
 'version': 'chatglm_turbo',
 'worker_class': <class 'server.model_workers.zhipu.ChatGLMWorker'>}
{'api_base_url': 'https://api.openai.com/v1',
 'api_key': '',
 'device': 'auto',
 'host': '0.0.0.0',
 'infer_turbo': False,
 'model_name': 'gpt-3.5-turbo',
 'online_api': True,
 'openai_proxy': '',
 'port': 20002}
当前Embbedings模型： bge-large-zh